In [1]:
%run ../scripts/notebook_settings.py
import sgkit as sg
import xarray as xr
import glob

Workflow to load in the respective individuals and generate base statistics on them.

In [2]:
# Functions

def read_beds(long_form):
    bed_path_x = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024/{}/filteredVCF/pos_bed_cov_based/{}_batch*_fploidy2_mploidy1.bed".format(long_form, long_form)
    bed_path_all = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024/{}/filteredVCF/pos_bed_cov_based/{}_batch*_fploidy2_mploidy2.bed".format(long_form, long_form)
    bed_l = []
    for b in glob.glob(bed_path_all):
        bed_file = pd.read_csv(b, sep="\t", names=["chrom", "start", "end"])
        bed_l.append(bed_file)
    bed_files = pd.concat(bed_l)
    bed_l = []
    for b in glob.glob(bed_path_x):
        #print(b)
        bed_file = pd.read_csv(b, sep="\t", names=["chrom", "start", "end"])
        bed_l.append(bed_file)
    if len(bed_l) > 0:
        bed_x = pd.concat(bed_l)
        bed_files = bed_files.loc[~(bed_files.chrom.isin(bed_x.chrom.unique()))]
        bed_files = pd.concat([bed_files, bed_x]).sort_values(by=["chrom", "start", "end"])
    return bed_files

def pos_windows(bed_l, window_size, chrom_order):
    # Input a bed file and the window size of intervals desired. Multiple chromosomes accepted.
    # It has to be sorted.
    df_l = []
    for c in chrom_order:
        #print(c)
        frac_l = []
        b = bed_l.loc[bed_l["chrom"] == c].copy()
        b["w_s"] = b.end-b.start
        w_start = b.start.iloc[0]
        current_pos, callable_bases = 0, 0
        for i, j, k in zip(b.start, b.end, b.w_s):
            # Nothing called in the current window under investigation.
            while i-window_size >= current_pos:
                frac_l.append(callable_bases/window_size)
                callable_bases = 0
                current_pos += window_size
            # Window starts in current. We know this is true because of the previous while loop.
            callable_bases += min(k, current_pos+window_size-i)
            # Everything called in current.
            while j-window_size >= current_pos:
                frac_l.append(callable_bases/window_size)
                callable_bases = 0
                current_pos += window_size
                if j-window_size >= current_pos:
                    callable_bases += window_size
                else:
                # Window stops in current. Again, know this is true.
                    callable_bases += j-current_pos
        # Last window.
        frac_l.append(callable_bases/(window_size))
        df_l.append(pd.DataFrame({"chrom": c, "window_start": list(range(0, len(frac_l)*window_size, window_size)),
                                  "window_end": list(range(window_size, (len(frac_l)+1)*window_size, window_size)),
                                  "callable_frac": frac_l}))
    return pd.concat(df_l)

def haploid_double(ds, variable, dim):
    unmasked = ~ds[f"{variable}_mask"]
    overwrite = ds.call_genotype[:,:,0]
    overwrite_2 = ds.call_genotype_mask[:,:,0]
    return ds.assign(**{
        f"{variable}": ds[variable].where(
        unmasked, 
        overwrite),
        f"{variable}_mask": ds[f"{variable}_mask"].where(
        unmasked,
        overwrite_2)})

In [3]:
metadata_path = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/"
zarr_path = "../zarr_data/"
metadata_folders = glob.glob(metadata_path+"*_individuals.txt")

size_cutoff = 1000000
window_size = 100000
missing_filter = 0.5

In [4]:
glob.glob("../zarr_data/Macaca_maura_ssp/*")

['../zarr_data/Macaca_maura_ssp/NC_092145.1',
 '../zarr_data/Macaca_maura_ssp/NW_027257673.1',
 '../zarr_data/Macaca_maura_ssp/NC_092146.1',
 '../zarr_data/Macaca_maura_ssp/NC_092125.1']

In [5]:

print("Loading metadata")
zarr_path = "../zarr_data/Macaca_maura_ssp"
metadata_path = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/"

short_form = zarr_path.split("/")[-1].split("_")[0]
long_form = zarr_path.split("/")[-1]
# Loading the various metadata files. Metadata, contig information, callability bed.
metadata_df = pd.read_csv(metadata_path+"{}_individuals.txt".format(short_form), sep="\t")
metadata_df["SEX_I"] = [0 if x == "F" else 1 for x in metadata_df.GENETIC_SEX]
regions_df = pd.read_csv(metadata_path+"{}_regions_and_batches.txt".format(short_form), sep="\t")
regions_df["LENGTH"] = regions_df["END"]-regions_df["START"]
regions_df["chr_type"] = ["chrX" if x == 2 and y == 1 else "aut" for x, y in zip(regions_df.FEMALE_PLOIDY, regions_df.MALE_PLOIDY)]
large_contigs = regions_df.loc[(regions_df.LENGTH >= 1000000) & (regions_df.FEMALE_PLOIDY == 2)].CONTIG_ID.unique()
large_x = regions_df.loc[(regions_df.LENGTH >= 1000000) & (regions_df.FEMALE_PLOIDY == 2) &
                        (regions_df.MALE_PLOIDY == 1)].CONTIG_ID
print("Loading bed files")
bed_files = read_beds(long_form)
# Loading the genetic data.
print("Loading genetic data")
df_l = []
for c in glob.glob(zarr_path+"/*"):
    ds = sg.load_dataset(c)
    print(c)

Loading metadata
Loading bed files
Loading genetic data
../zarr_data/Macaca_maura_ssp/NC_092145.1
../zarr_data/Macaca_maura_ssp/NW_027257673.1
../zarr_data/Macaca_maura_ssp/NC_092146.1
../zarr_data/Macaca_maura_ssp/NC_092125.1


In [6]:
ds_var_stats = sg.variant_stats(ds)

In [7]:
ds_var_stats.variant_n_hom_alt[:20].compute()

<xarray.DataArray 'variant_n_hom_alt' (variants: 20)> Size: 160B
array([0, 0, 0, 2, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
Dimensions without coordinates: variants

In [8]:
from sgkit.cohorts import _cohorts_to_array
from sgkit.stats.utils import assert_array_shape
from sgkit.utils import (
    conditional_merge_datasets,
    create_dataset,
    define_variable_if_absent,
)
from sgkit.window import has_windows, window_statistic

from sgkit import variables
from sgkit.stats.aggregation import (
    count_cohort_alleles,
    count_variant_alleles,
    individual_heterozygosity,
)

In [ ]:
def alt_hom(ds):
    alt_hom = ((ds.call_genotype[:,:,0] == ds.call_genotype[:,:,1]) & (ds.call_genotype[:,:,0] >= 1)).sum(axis=1)
    div = window_statistic(
            alt_hom,
            np.sum,
            ds.window_start.values,
            ds.window_stop.values,
            dtype=np.int64,
            axis=0,
    )
    return div

In [ ]:
ds["sample_cohort"] = ds["samples"]
ds = ds.sel(contigs=[ds.variant_contig[0].values])
ds = sg.window_by_position(ds, size=100000)
alt_hom(ds)[:10].values

In [9]:
ds

<xarray.Dataset> Size: 150MB
Dimensions:               (variants: 2023479, samples: 7, ploidy: 2,
                           contigs: 361, filters: 2, region_index_values: 2024,
                           region_index_fields: 6, alleles: 2)
Dimensions without coordinates: variants, samples, ploidy, contigs, filters,
                                region_index_values, region_index_fields,
                                alleles
Data variables: (12/16)
    call_genotype         (variants, samples, ploidy) int8 28MB dask.array<chunksize=(1000, 7, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool 28MB dask.array<chunksize=(1000, 7, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool 14MB dask.array<chunksize=(1000, 7), meta=np.ndarray>
    contig_id             (contigs) object 3kB dask.array<chunksize=(361,), meta=np.ndarray>
    contig_length         (contigs) float64 3kB dask.array<chunksize=(361,), meta=np.ndarray>
    filter_id             (filters) object 16B dask.array<chunksize=(2,), meta=np.ndarray>
    ...                    ...
    variant_filter        (variants, filters) bool 4MB dask.array<chunksize=(1000, 2), meta=np.ndarray>
    variant_id            (variants) object 16MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_id_mask       (variants) bool 2MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_length        (variants) int16 4MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_position      (variants) int32 8MB dask.array<chunksize=(1000,), meta=np.ndarray>
    variant_quality       (variants) float32 8MB dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes:
    source:            bio2zarr-0.1.5
    vcf_header:        ##fileformat=VCFv4.2\n##FILTER=<ID=PASS,Description="A...
    vcf_zarr_version:  0.2

In [13]:
#Count alt hom
((ds.call_genotype[:,:,0] == ds.call_genotype[:,:,1]) & (ds.call_genotype[:,:,0] >= 1)).sum(axis=1)[:10].values

array([0, 0, 0, 2, 5, 2, 0, 0, 0, 0])

In [14]:
#Count ref hom
((ds.call_genotype[:,:,0] == ds.call_genotype[:,:,1]) & (ds.call_genotype[:,:,0] == 0)).sum(axis=1)[:10].values

array([2, 2, 2, 0, 0, 0, 6, 3, 3, 5])

In [22]:
#Count all hom
((ds.call_genotype[:,:,0] == ds.call_genotype[:,:,1]) & (ds.call_genotype[:,:,0] >= 0)).sum(axis=1)[:20].values

array([2, 2, 2, 2, 5, 2, 6, 3, 3, 5, 5, 3, 0, 0, 0, 1, 2, 0, 2, 6])

In [42]:

ds = define_variable_if_absent(
        ds, variables.cohort_allele_count, cohort_allele_count, count_cohort_alleles
    )


NameError: name 'Hashable' is not defined

In [31]:
define_variable_if_absent

<function sgkit.utils.define_variable_if_absent(ds: xarray.core.dataset.Dataset, default_variable_name: Hashable, variable_name: Optional[Hashable], func: Callable[[xarray.core.dataset.Dataset], xarray.core.dataset.Dataset], **kwargs) -> xarray.core.dataset.Dataset>

In [ ]:
# Fst implementation.
for x in glob.glob(zarr_path+"*")[25:]:
    # The name used to load all the files, short and long version
    short_form = x.split("/")[-1].split("_")[0]
    long_form = x.split("/")[-1]
    # Loading the various metadata files. Metadata, contig information, callability bed.
    metadata_path = "/home/eriks/primatediversity/data/gVCFs_recalling_10_12_2024_metadata/"
    metadata_df = pd.read_csv(metadata_path+"{}_individuals.txt".format(short_form), sep="\t")
    metadata_df["SEX_I"] = [0 if x == "F" else 1 for x in metadata_df.GENETIC_SEX]
    regions_df = pd.read_csv(metadata_path+"{}_regions_and_batches.txt".format(short_form), sep="\t")
    regions_df["LENGTH"] = regions_df["END"]-regions_df["START"]
    regions_df["chr_type"] = ["ChrX" if x == 2 and y == 1 else "ChrY" if x == 0 and y == 1 else "Aut" for x, y in zip(regions_df.FEMALE_PLOIDY, regions_df.MALE_PLOIDY)]
    large_contigs = regions_df.loc[(regions_df.LENGTH >= size_cutoff) & (regions_df.FEMALE_PLOIDY == 2)].CONTIG_ID.unique()
    large_x = regions_df.loc[(regions_df.LENGTH >= size_cutoff) & (regions_df.FEMALE_PLOIDY == 2) &
                        (regions_df.MALE_PLOIDY == 1)].CONTIG_ID
    # Skipping the large samples sizes and the singulars for the first Fst calc
    # Also perform a 10X filter to get accurate samples sizes
    sample_size = len(metadata_df.loc[(metadata_df.GVCF_FOLDER == long_form) &
                                     (metadata_df.AVG_COVERAGE_A >= 10)])
    print(sample_size, long_form)
    if (sample_size > 20) or (sample_size <= 1):
        print("Skipping, sample size", sample_size)
        continue
    if os.path.exists("../results/window_stats/{}_Fst.txt".format(long_form)):
        print("Skipping as it's processed")
        continue
    if len(glob.glob(x+"/*")) == 0:
        print("Skipping, no completed zarrs")
        continue
    # Loading the genetic data.
    df_l = []
    for c in glob.glob(x+"/*"):
        print(c)
        ds = sg.load_dataset(c)
        # This implementation is the pi implementation.
        # Probably problematic in some cases with population structure, but it is easier to implement
        ds["sample_cohort"] = ds["samples"]
        # Subsetting and windowing the sgkit dataset. The rechunking handles what otherwise would cause an error.
        #ds["call_genotype"] = ds["call_genotype"].clip(0)
        ds = ds.sel(contigs=[ds.variant_contig[0].values])
        if c.split("/")[-1] in list(large_x):
            ds = haploid_double(ds, "call_genotype", "samples")
        missing_rate = ds.call_genotype_mask[:,:,0].sum(axis=1).values/ds.call_genotype_mask[:,:,0].count(axis=1).values
        ds = ds.isel(variants=(missing_rate <= 0))
        ds = sg.window_by_genome(ds)
        ds = (sg.Fst(ds.chunk({"variants": 10000})))
        df_sub = pd.DataFrame(ds.stat_Fst[0,:,], columns=ds.sample_id)
        df_sub["chrom"] = c.split("/")[-1]
        df_sub["variants_used"] = len(ds.variants)
        df_sub["stat"] = "Fst"
        df_l.append(df_sub)
        df_sub = pd.DataFrame(ds.stat_divergence[0,:,], columns=ds.sample_id)
        df_sub["chrom"] = c.split("/")[-1]
        df_sub["variants_used"] = len(ds.variants)
        df_sub["stat"] = "Divergence"
        df_l.append(df_sub)
    output_df = pd.concat(df_l)
    output_df["chr_type"] = output_df["chrom"].map(dict(zip(regions_df.CONTIG_ID, regions_df.chr_type)))
    output_df["species"] = long_form
    output_df.to_csv("../results/window_stats/{}_Fst.txt".format(long_form), sep="\t")

2 Avahi_laniger_ssp
../zarr_data/Avahi_laniger_ssp/HiC_scaffold_2
../zarr_data/Avahi_laniger_ssp/HiC_scaffold_1
1 Lepilemur_sahamalazensis_ssp
Skipping, sample size 1
3 Allenopithecus_nigroviridis_ssp
../zarr_data/Allenopithecus_nigroviridis_ssp/HiC_scaffold_1
../zarr_data/Allenopithecus_nigroviridis_ssp/HiC_scaffold_2
3 Alouatta_belzebul_ssp
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000017.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000080.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000046.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000034.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000072.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000001.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000040.1
../zarr_data/Alouatta_belzebul_ssp/CAJZLT010000060.1
2 Cercopithecus_denti_ssp
../zarr_data/Cercopithecus_denti_ssp/CM053398.1
../zarr_data/Cercopithecus_denti_ssp/CM053363.1
10 Saimiri_ustus_ssp
../zarr_data/Saimiri_ustus_ssp/NW_024100917.1
../zarr_data/Saimiri_ustus_ssp/NW_02410092

../zarr_data/Alouatta_juara_ssp/CAJZLT010000072.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000040.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000060.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000017.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000046.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000080.1
../zarr_data/Alouatta_juara_ssp/CAJZLT010000034.1
1 Hylobates_klossii_ssp
Skipping, sample size 1
4 Cheracebus_lucifer_ssp
../zarr_data/Cheracebus_lucifer_ssp/CM080837.1
../zarr_data/Cheracebus_lucifer_ssp/CM080815.1
1 Propithecus_deckenii_ssp
Skipping, sample size 1
2 Cheracebus_regulus_ssp
../zarr_data/Cheracebus_regulus_ssp/CM080815.1
../zarr_data/Cheracebus_regulus_ssp/CM080837.1
2 Alouatta_palliata_ssp
../zarr_data/Alouatta_palliata_ssp/CAJZLT010000017.1
../zarr_data/Alouatta_palliata_ssp/CAJZLT010000034.1
../zarr_data/Alouatta_palliata_ssp/CAJZLT010000046.1
../zarr_data/Alouatta_palliata_ssp/CAJZLT010000080.1
../zarr_data/Alouatta_palliata_ssp/CAJZLT010000001.1
../zarr_data/Alouatta_pa

../zarr_data/Papio_papio_ssp/NC_044976.1
3 Galago_senegalensis_ssp
../zarr_data/Galago_senegalensis_ssp/HiC_scaffold_1
../zarr_data/Galago_senegalensis_ssp/HiC_scaffold_31
2 Ateles_paniscus_ssp
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000040.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000060.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000072.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000001.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000080.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000046.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000034.1
../zarr_data/Ateles_paniscus_ssp/CAJZLT010000017.1
5 Varecia_variegata_ssp
../zarr_data/Varecia_variegata_ssp/CM052457.1
../zarr_data/Varecia_variegata_ssp/CM052436.1
1 Cercocebus_lunulatus_ssp
Skipping, sample size 1
1 Macaca_siberu_ssp
Skipping, sample size 1
2 Xanthonycticebus_pygmaeus_ssp
../zarr_data/Xanthonycticebus_pygmaeus_ssp/NC_069780.1
../zarr_data/Xanthonycticebus_pygmaeus_ssp/NC_069804.1
../zarr_data/Xanthonycticebus_pygmaeus_ssp/NC

../zarr_data/Macaca_leonina_ssp/NC_092145.1
../zarr_data/Macaca_leonina_ssp/NC_092146.1
../zarr_data/Macaca_leonina_ssp/NC_092125.1
7 Pygathrix_nemaeus_ssp
../zarr_data/Pygathrix_nemaeus_ssp/NC_044549.1
../zarr_data/Pygathrix_nemaeus_ssp/NC_044555.1
3 Otolemur_crassicaudatus_ssp
../zarr_data/Otolemur_crassicaudatus_ssp/HiC_scaffold_1
3 Cercopithecus_cephus_ssp
../zarr_data/Cercopithecus_cephus_ssp/CM053398.1
../zarr_data/Cercopithecus_cephus_ssp/CM053363.1
2 Loris_lydekkerianus_ssp
../zarr_data/Loris_lydekkerianus_ssp/NC_069805.1
../zarr_data/Loris_lydekkerianus_ssp/NC_069804.1
../zarr_data/Loris_lydekkerianus_ssp/NC_069780.1
80 Papio_cynocephalus_ssp
Skipping, sample size 80
1 Carlito_syrichta_ssp
Skipping, sample size 1
21 Alouatta_seniculus_ssp
Skipping, sample size 21
13 Cercocebus_atys_ssp
../zarr_data/Cercocebus_atys_ssp/HiC_scaffold_21
../zarr_data/Cercocebus_atys_ssp/HiC_scaffold_111
../zarr_data/Cercocebus_atys_ssp/HiC_scaffold_1
17 Pan_paniscus_ssp
../zarr_data/Pan_paniscus_s

In [27]:
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


In [46]:
def divergence_divide(square_df):
    square_out = np.zeros((len(square_df), len(square_df)))
    for i in range(len(square_df)):
        for j in range(len(square_df)):
            square_out[i,j] = square_df.iloc[i,j]/((square_df.iloc[i,i]+square_df.iloc[j,j])/2)
    return square_out

Flexible intervals:

In [ ]:
def interval_creator(bed_l, window_size):
    # Input a bed file and the window size of intervals desired. Multiple chromosomes accepted.
    df_l = []
    for c in bed_l.chrom.unique():
        print(c)
        start_l, end_l = [], []
        b = bed_l.loc[bed_l["chrom"] == c].copy()
        b["w_s"] = b.end-b.start
        w_start = b.start.iloc[0]
        current_size = 0
        for i, j, k in zip(b.start, b.end, b.w_s):
            # Current window encapsulates the final stretch of the interval.
            if current_size + k >= window_size:
                start_l.append(w_start), end_l.append(i+(window_size-current_size))
                w_start = i+(window_size-current_size)
                # If the window still contains full intervals, contigous windows until it cant.
                for x in range((k-window_size+current_size)//window_size):
                    start_l.append(w_start), end_l.append(w_start+window_size)
                    w_start += window_size
                current_size = j-w_start
            # Current window does not encapsulate per definition, so it has to be added to current size but nothing else.
            else:
                current_size += k
        df_l.append(pd.DataFrame({"chrom": c, "interval_start": start_l, "interval_end": end_l}))
    return pd.concat(df_l)